Bây giờ chúng ta đã biết cách truy cập các tham số, hãy xem cách khởi tạo chúng đúng cách. Chúng ta đã thảo luận về nhu cầu khởi tạo thích hợp trong Phần 5.4 . Khung học sâu cung cấp các khởi tạo ngẫu nhiên mặc định cho các lớp của nó. Tuy nhiên, chúng tôi thường muốn khởi tạo trọng số của mình theo nhiều giao thức khác. Khung cung cấp các giao thức được sử dụng phổ biến nhất và cũng cho phép tạo trình khởi tạo tùy chỉnh.

In [1]:
import torch
from torch import nn

Theo mặc định, PyTorch khởi tạo ma trận trọng số và độ chệch một cách thống nhất bằng cách vẽ từ một phạm vi được tính toán theo thứ nguyên đầu vào và đầu ra. Mô-đun của PyTorch `nn.init` cung cấp nhiều phương thức khởi tạo đặt trước.

In [5]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

# 6.3.1. Khởi tạo tích hợp

Hãy bắt đầu bằng cách gọi các trình khởi tạo tích hợp sẵn. Đoạn mã dưới đây khởi tạo tất cả các tham số trọng số dưới dạng các biến ngẫu nhiên Gaussian với độ lệch chuẩn 0,01, trong khi các tham số sai lệch được xóa bằng 0.

In [6]:
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01) # Phân bố chuẩn
        nn.init.zeros_(module.bias)                      # 0

net.apply(init_normal) #Hàm apply dùng ntn ?!
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0053, -0.0063, -0.0119,  0.0093]), tensor(0.))

Chúng ta cũng có thể khởi tạo tất cả các tham số thành một giá trị không đổi nhất định (giả sử là 1).

In [7]:
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1) #Hằng số
        nn.init.zeros_(module.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

Chúng ta cũng có thể áp dụng các trình khởi tạo khác nhau cho các khối nhất định. Ví dụ: bên dưới, chúng tôi khởi tạo lớp đầu tiên bằng trình khởi tạo Xavier và khởi tạo lớp thứ hai thành giá trị không đổi là 42.

In [8]:
def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)

def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([0.5306, 0.2078, 0.2799, 0.2158])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


## 6.3.1.1. Khởi tạo tùy chỉnh

Đôi khi, các phương thức khởi tạo mà chúng ta cần không được cung cấp bởi deep learning framework. Trong ví dụ bên dưới, chúng tôi xác định một trình khởi tạo cho bất kỳ tham số trọng số nào $w$ sử dụng phân phối lạ sau:
$$
\begin{split}\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U(-10, -5) & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}\end{split}
$$

In [12]:
def my_init(module):
    if type(module) == nn.Linear:
        print("Init", *[(name, param.shape, param.data)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5

net.apply(my_init)
net[2].weight.data


Init weight torch.Size([8, 4]) tensor([[ 0.0000,  0.0000, -0.0000,  0.0000],
        [-5.7125,  7.6857,  5.4249,  7.5800],
        [-0.0000, -5.4866, -0.0000,  0.0000],
        [ 0.0000, -6.9298,  6.9753, -0.0000],
        [-0.0000, -6.5194,  0.0000,  0.0000],
        [ 0.0000, -8.3588, -0.0000,  9.3431],
        [ 9.7900,  7.9336, -0.0000, -0.0000],
        [ 7.9002, -8.0198, -0.0000, -7.7317]])
Init weight torch.Size([1, 8]) tensor([[ 0.0000,  0.0000, -9.0018, -0.0000,  6.1947, -5.9028,  7.8316, -6.9624]])


tensor([[-8.1302, -0.0000,  0.0000, -0.0000,  9.3347,  0.0000,  7.1990, -9.8849]])